# **Lecture 11. Kernels**
## **Applied Machine Learning**

## **Part 1: The Kernel Trick: Motivation**
So far, the majority machine learning models we have seen have been linear.

In this lecture, we will see a general way to make any of these models *non-linear*. We will use a new idea called *kernel*. 

### **Review: Linear Regression**
Recall that, a linear model has the form:
$$f_{\theta}(x) = \sum_{i=1}^d\theta_ix_i = \theta^\top x .$$

where $x$ is a vector of features and we use notation $x_0=1$.

We pick $\theta$ to minimize the (L2 regularized) mean squared errors (MSE):
$$J(\theta) = \frac{1}{2n}\sum_{i=1}^n(y^{(i)} - \theta^\top x^{(i)})^2 + \frac{\lambda}{2}\sum_{j=1}^d\theta_j^2.$$

### **Review: Polynomials**

Recall that a polynomial of degree $p$ is a function of the form:
$$a_px^p + a_{p-1}x^{p-1} + \dots + a_1x + a_0.$$

Below are some examples of polynomial functions.

### **Review: Polynomial Regression**

Specifically, given a one-dimensional continuous variable $x$, we can defining a feature function $\phi: \mathbb{R} \to \mathbb{R}^{p+1}$ as:

$$ \phi(x) = \begin{bmatrix}
1 \\
x \\
x^2 \\
\vdots \\
x^p
\end{bmatrix}.
$$

The class of the models of form:
$$f_{\theta}(x) := \sum_{j=0}^p\theta_jx^j = \theta^\top \phi(x).$$

with parameters $\theta$ and polynomial feature $\phi$ is the set of $p$-degree polynomial.

### **Towards General Non-linear Features**

Any non-linear feature map $\phi(x): \mathbb{R}^d \to \mathbb{R}^p$ can be used to obtain general models of the form:
$$f_{\theta}(x) := \theta^\top \phi(x)$$

that are highly non-linear in feature $x$ but linear in parameter $\theta$.



### **The Featurized Design Matrix**

It is useful to represent the featurized dataset as a matrix $\Phi \in \mathbb{R}^{n \times p}$
$$ \Phi = \begin{bmatrix}
\phi(x^{(1)})_1 & \phi(x^{(1)})_2 & \ldots & \phi(x^{(1)})_p \\
\phi(x^{(2)})_1 & \phi(x^{(2)})_2 & \ldots & \phi(x^{(2)})_p \\
\vdots \\
\phi(x^{(n)})_1 & \phi(x^{(n)})_2 & \ldots & \phi(x^{(n)})_p
\end{bmatrix}
=
\begin{bmatrix}
- & \phi(x^{(1)})^\top & - \\
- & \phi(x^{(2)})^\top & - \\
& \vdots & \\
- & \phi(x^{(n)})^\top & - \\
\end{bmatrix}
.$$

### **Featurized Normal Equations**

The normal equations provide a closed-form solution for $\theta$:

$$\theta = (X^\top X + \lambda I)^{-1}X^\top y.$$

When the vector of attributes $x^{(i)}$ are featurized, we can write this as:
$$\theta = (\Phi^\top\Phi + \lambda I)^{-1}\Phi^\top y.$$

### **Push-Through Matrix Identity**

We can modify this expression by using a version of the [push-through matrix identity](https://en.wikipedia.org/wiki/Woodbury_matrix_identity#Discussion)
$$(\lambda I + UV)^{-1}U = U(\lambda I + VU)^{-1}$$

where $U \in \mathbb{R}^{n\times m}$ and $V \in \mathbb{R}^{m \times n}$ and $\lambda \neq 0$

Proof sketch: Start with $U(\lambda I + VU) = (\lambda I + UV)U$ and multiply both sides by $(\lambda I + VU)^{-1}$ on the right and $(\lambda I + UV)^{-1}$ on the left.

### **Normal Equations: Dual Form**
We can apply the identity $(\lambda I + UV)^{-1} U = U(\lambda I + VU)^{-1}$ to the normal equation with $U = \Phi^\top, V = \Phi.$

$$\theta = (\Phi^\top\Phi + \lambda I)^{-1}\Phi^\top y.$$

to obtain a dual form:
$$\theta = \Phi^\top(\Phi\Phi^\top + \lambda I)^{-1}y$$

This approach take $O(p^3)$ times; the second is $O(n^3)$ times and is faster when $p>n.$

### **Feature Representation for Parameters**
An interesting corollary of the dual form:
$$\theta = \Phi^\top\underbrace{(\Phi\Phi^\top + \lambda I)^{-1}y}_{\alpha}$$

is that the optimal $\theta$ is a linear combination of the n training set features:
$$\theta = \sum_{i=1}^n\alpha_i\phi(x^{(i)}).$$

Here, the weights $\alpha $ derived from $(\Phi\Phi^\top + \lambda I)^{-1}y$ and equals:
$$\alpha_i = \sum_{j=1}^nL_{ij}y_j$$

where $L_{ij} = (\Phi^\top \Phi + \lambda I)^{-1}.$

### **Predictions from Features**
Consider now a prediction $\phi(x')^\top \theta$ at a new input $x'$
$$\phi(x')^\top\theta = \sum_{i=1}^n\alpha_i\phi(x')^\top\phi(x^{(i)}).$$

The crucail observation is that the features $\phi(x)$ are never used directly in this equation. Only their dot product is used!

This observation will be at the heart of a powerful new idea called the *kernel trick.*

### **Learning from the Feature Products**

We also don't need features $\phi$ for learning $\theta$, just their dot products! Recall that each row $i$ of $\Phi$ is the $i$-th featurized input $\phi(x^{(i)})^\top.$

Thus $K = \Phi\Phi^\top$ is a matrix of all dot products between all the $\phi(x^{(i)})$
$$K_{ij} = \phi(x^{(i)})^\top\phi(x^{(j)}).$$

We can compute $\alpha = (K + \lambda I)^{-1}y$ and uses it for predictions
$$\phi(x')^\top\theta = \sum_{i=1}^n\alpha_i\phi(x')^\top\phi(x^{(i)}).$$

and all this only requires dot products, not features $\phi$!

### **The Kernel Trick**

The above observations hint at a powerful new idea -- if we can compute dot products of features $\phi(x)$ efficiently, then we will be able to use high-dimensional features easily.

It turns out that we can do this for many ML algorithms -- we call this the **Kernel Trick**

## **Part 2: The Kernel Trick: An Example**

Many ML algorithms can be written down as optimization problems in which the feature $\phi(x)$ only appears in the dot products form $\phi(x)^\top \phi(z)$ that can be computed efficiently.

Let's look at an example:

### **Review: Non-linear Features**

Any non-linear feature map $\phi(x): \mathbb{R}^d \to \mathbb{R}^p$ can be used in this way to obtain the general model of form:
$$f_{\theta}(x) := \theta^\top x.$$

that are highly non-linear in the feature $x$ but linear in $\theta$.

### **Review: Normal Equations**

The normal equations provide a closed form solution for $\theta$:
$$\theta = (\Phi^\top\Phi + \lambda I)\Phi^\top y.$$

They can also be written in this form:
$$\theta = \Phi^\top(\Phi\Phi^\top + \lambda I) y.$$

The first approach takes $O(d^3)$ times, and the second takes $O(n^3)$, and is faster when $d>n$

### **Learning from Feature Products**

An interesting corollary is that the optimal $\theta$ is a linear combination of the $n$ training set features:
$$\theta = \sum_{i=1}^n\alpha_i\phi(x^{(i)})).$$

We can compute a prediction $\phi(x')^\top\theta$ for $x'$ without ever using the features (only their dot products).

$$\phi(x')^\top x = \sum_{i=1}^n \alpha_i\phi(x')^\top\phi(x^{(i)}). $$

Equally importantly, we can learn $\theta$ form only dot products.


### **Review: Polynomial Regression**

Note that a $p$-th degree polynomial

$$a_px^p + a_{p-1}x^{p-1} + \dots + a_1x + a_0$$

forms a linear model with parameters $a_p, a_{p-1}, a_{p-2}, \dots, a_1, a_0$. This means we can use our algorithms for linear models to learn non-linear features.

specifically, given a one-dimensional continuous variable $x$, we can define the feature function $\phi: \mathbb{R} \to \mathbb{R^p}$ as

$$
\phi(x) = \begin{bmatrix}
1 \\
x \\
x^2 \\
\vdots \\
x^p
\end{bmatrix}.
$$

Then the class of the model of the form:
$$f_{\theta}(x) := \sum_{i=0}^p\theta_i x^i =\theta^\top \phi(x)$$

with parameters $\theta$ encompasses the set of $p$-degree polynomial. Specifically
* It is non-linear in the input variable $x$, meaning that we can model complex data relationship.
* It is a linear model of the function of parameters $\theta$, meaning that we can use our familiar ordinary least squares algorithm to learn thees features.

### **The Kernel Trick: A First Example**
Can we compute the dot products of $\phi(x)^\top\phi(x')$ of the polynomial $\phi(x)$ more efficiently than using the standard definition of a dot product? 

Let's look at an example

To start, let's consider polynomial features  $\phi: \mathbb{R}^d \to \mathbb{R}^{d^2}$ of the form:
$$\phi(x)_{ij} = x_ix_j \; \text{for} \; i,j \in \{1,2,\dots,d\}$$

For $d=3$ this looks like:

$$
\phi(x) = \begin{bmatrix}
x_1x_1\\
x_1x_2\\
x_1x_3\\
x_2x_1\\
x_2x_2\\
x_2x_3\\
x_3x_1\\
x_3x_2\\
x_3x_3
\end{bmatrix}.
$$



The product of $x$ and $z$ in the feature space equals:

$$\phi(x)^\top\phi(z) = \sum_{i=1}^d\sum_{j=1}^dx_ix_jz_iz_j.$$

Computing this products involves the sum over $d^2$ terms and takes $O(d^2)$ times.

An alternative way of computing the product of $\phi(x)^\top\phi(z)$ is to instead compute $(x^\top z)^2$. We can check that this has the same result:

\begin{align*}
(x^\top z)^2 &= (\sum_{i=1}^dx_iz_i)^2 \\
&= (\sum_{i=1}^dx_iz_i)(\sum_{j=1}^dx_jz_j) \\
&= \sum_{i=1}^d\sum_{j=1}^dx_ix_jz_iz_j \\
&= \phi(x)^\top\phi(z)
\end{align*}

However, computing $(x^\top z)^2$ can be done in $O(d)$ times.

This is a very powerful idea.
* We can compute the dot products between $O(d^2)$ features in only $O(d)$ times.
* We can use high-dimensional features within the ML algorithms the only rely on the dot products (like kernelized ridge regression) without incurring extra costs.

### **The Kernal Trick: Polynomial Features**
The number of polynomial features $\phi_p$ of $p$-degree when $x \in \mathbb{R}^d$

$$\phi(x)_{i_1,i_2,\dots,i_p} = x_{i_1}x_{i_2}\dots x_{i_p} \; \text{for} \; i_1, i_2, \dots, i_p \in \{1,2,\dots,d\}$$

scales as $O(d^p)$.

However, we can compute the dot product $\phi_p(x)^\top\phi_p(x)$ in this feature space in only $O(d)$ times for any $p$ as:
$$\phi_p(x)^\top\phi_p(x) = (x^\top z)^p.$$

### **Algorithm: Kenerlised Polynomial Ridge Regression** 
* Type: Supervised Learning (regression)
* Model family: Polynomials
* Objective function: $L2$-regularized ridge regression
* Optimizer: Normal equations (dual forms)
 

### **The Kernel Trick: General Idea**

Many types of feature $\phi(x)$ have the property that their dot product $\phi(x)^\top\phi(z)$ can be computed more efficiently than if we had to form these features explicitly.

Also, we will see many ML algorithms can be written down as optimization problem in which the features $\phi(x)$ only appear as dot product $\phi(x)^\top\phi(z)$.

The *Kernel Trick* means that we can use complex non-linear features within these algorithms with little computational cost.

Examples of algorithms in which we can use the kernel trick:
* Supervised learning algorithms: linear regression, logistic regression, support vector machine, etc.
* Unsupervised learning: PCA, density estimation.




We will look at more example shortly.

## **Part 3: The Kernel Trick in SVMs**

Many ML algorithms can be written down as optimization problems in which the features $\phi(x)$ only appear as dot products $\phi(x)^\top\phi(x)$ that can be computed efficiently.

We will now see how SVMs can benefit from the Kernel Trick as well.

### **Review: SVM Model Family**

We will consider models of the form:
$$f_{\theta}(x) = \theta^\top \phi(x) + \theta_0$$

where $x$ is the input and $y \in \{-1,+1\}$ is the target.

### **Review: Dual and Primal Formulations** 

Recall that the max-margin hyperplane can be formulated as the solution to the following *primal* optimization problem.

\begin{align*}
\min_{\theta,\theta_0,\xi}&\frac{1}{2}\|\theta\|^2 + C\sum_{i=1}^n\xi_i \\
\text{subjected to} \; &y^{(i)}((x^{(i)})^\top\theta + \theta_0) \geq 1 - \xi_i \; \text{for all} \; i \\
& \xi_i \geq 0
\end{align*}

The solution to this problem also happens to be given by the following *dual* problem:
\begin{align*}
\max_{\lambda}&\sum_{i=1}^n\lambda_i - \frac{1}{2}\sum_{i=1}^n\sum_{k=1}^n\lambda_i\lambda_ky^{(i)}y^{(k)}(x^{(i)})^\top x^{(k)}\\
\text{subjected to} \; &\sum_{i=1}^n\lambda_iy^{(i)} = 0 \\
&C \geq \lambda_i \geq 0 \; \text{for all} \; i
\end{align*}

### **Review: The Primal Solution**

We can obtain a primal solution from the dual via the following equation:

$$\theta^* = \sum_{i=1}^n\lambda_i^*y^{(i)}\phi(x^{(i)}).$$

Ignoring the $\theta_0$ term for now, the score at a new point $x'$ will be equal:
$$\theta^\top\phi(x') = \sum_{i=1}^n\lambda_i^*y^{(i)}\phi(x^{(i)})^\top\phi(x').$$ 

### **The Kernel Trick in SVMs**

Notice that in both equations, the feature $x$ are never used directly. Only their dot product is used
\begin{align*}
\sum_{i=1}^n\lambda_i - &\frac{1}{2}\sum_{i=1}^n\sum_{k=1}^n\lambda_i\lambda_ky^{(i)}y^{(k)}\phi(x^{(i)})^\top \phi(x^{(k)})\\ 
&\theta^\top\phi(x') = \sum_{i=1}^n\lambda_i^*y^{(i)}\phi(x^{(i)})^\top\phi(x')
\end{align*}

If we can compute the dot products efficiently, we can potentilly use very complex features.

### **The Kernel Trick in SVMs**
More generally, given features $\phi(x)$, suppose that we have a function $K: \mathcal{X} \times \mathcal{X} \to [0,\infty)$ that outputs dot products between vectors in $\mathcal{X}$.
$$K(x,z) = \phi(x)^\top\phi(z).$$

Will call $K$ is the kernel function.

Recall that an example of a useful kernel function is:
$$K(x,z) = (x\cdot z)^p$$

because it computes the dot product of polynomial features of degree $p$.

Then notice that we can rewrite the dual form of SVM as

\begin{align*}
\max_{\lambda}&\sum_{i=1}^n\lambda_i - \frac{1}{2}\sum_{i=1}^n\sum_{k=1}^n\lambda_i\lambda_ky^{(i)}y^{(k)}K(x^{(i)}, x^{(k)})\\
\text{subjected to} \; &\sum_{i=1}^n\lambda_iy^{(i)} = 0 \\
&C \geq \lambda_i \geq 0 \; \text{for all} \; i
\end{align*}

and *predictions* at new point $x'$ are given by $\sum_{i=1}^n\lambda_i^*y^{(i)}K(x^{(i)},x').$

Using our earlier trick, we can use polynomial features of any degree of $p$ in SVM without forming these features and not extra cost!

### **Algorithm: Kernelized Support Vector Machine classification (Dual Form)**

* **Type**: Supervised learning (binary classification)
* **Model Family**: Non-linear decision boundaries
* **Objective function**: Dual of SVM optimization problem
* **Optimizer**: Sequential Minimal optimization

## **Part 4: Types of Kernels**

Now that we saw the kernel trick, let's look at several examples of kernels.

### **Kernel Trick for Ridge Regression**

The normal equation provides a closed form solution for $\theta$:
$$\theta = (\Phi^\top\Phi + \lambda I)^{-1}\Phi^\top y.$$

They also can be written in the form:
$$\theta = \Phi^\top(\Phi\Phi^\top + \lambda I)^{-1}y.$$

The first approach takes $O(d^3)$ times, and the second takes $O(n^3)$ times and is faster if $d>n$.

An interesting corollary is that the optimal $\theta$ is a linear combination of the $n$ training set features:
$$\theta = \sum_{i=1}^n\alpha_i\phi(x^{(i)}).$$

We can compute a prediction $\phi(x')^\top\theta$ for a new point $x'$ without ever using the features (only their dot products).
$$\phi(x')^\top\theta = \sum_{i=1}^n\alpha_i\phi(x')^\top\phi(x^{(i)})$$

Equally importantly, we can learn $\theta$ from only dot products.


### **Review: Kernel Trick in SVMs**

Notice that in both equations, the feature $x$ are never used directly. Only their dot product is used
\begin{align*}
\sum_{i=1}^n\lambda_i - &\frac{1}{2}\sum_{i=1}^n\sum_{k=1}^n\lambda_i\lambda_ky^{(i)}y^{(k)}\phi(x^{(i)})^\top \phi(x^{(k)})\\ 
&\theta^\top\phi(x') = \sum_{i=1}^n\lambda_i^*y^{(i)}\phi(x^{(i)})^\top\phi(x')
\end{align*}

If we can compute the dot products efficiently, we can potentilly use very complex features.

### **Definition: Kernels**

The kernel corresponding to the features $\phi(x)$ is the function $K: \mathcal{X} \times \mathcal{X} \to [0,\infty)$ that outputs the dot products between vectors in $\mathcal{X}$.

$$K(x,z) = \phi(x)^\top \phi(z).$$

We will also consider general functions $K:\mathcal{X} \times \mathcal{X} \to [0,\infty)$ and call these *kernel function*.

Kernels have many interpretations:
* The dot product or geometrical angle between $x$ and $z$,
* A notion of similarity between $x$ and $z$.



in order to illustrate kernels, we will use this dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm

# Our dataset and target.
X = np.c_[
    (0.4, -0.7),
    (-1.5, -1),
    (-1.4, -0.9),
    (-1.3, -1.2),
    (-1.1, -0.2),
    (-1.2, -0.4),
    (-0.5, 1.2),
    (-1.5, 2.1),
    (1, 1),
    # --
    (1.3, 0.8),
    (1.2, 0.5),
    (0.2, -2),
    (0.5, -2.4),
    (0.2, -2.3),
    (0, -2.7),
    (1.3, 2.1),
].T

y = [0]*8 + [1]*8

x_min, x_max = -3, 3
y_min, y_max = -3, 3
plt.scatter(X[:,0], X[:,1], c=y, zorder=10, cmap=plt.cm.Paired, edgecolors='k')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)


### **Linear Kernel**

The simplest kind of kernel that exists is called the linear kernel. This simply corresponds to dot product multiplication of the feature:

$$K(x,z) = x^\top z$$

Applied to an SVM, this corresponds to a linear decision boundary.

Below is an example of how we can use the SVM inplementation in `sklearn` with a linear kernel.

Internally, this solves the dual SVM optimization problem.

In [ ]:
clf = svm.SVC(kernel='linear', gamma=2)
clf.fit(X,y)

# Plot the line, the points and the nearest vectors to the plane
plt.scatter(clf.support_vectors_[:,0], clf.support_vectors_[:,1], s=80, facecolors='none', edgecolors='k', zorder=10)
plt.scatter(X[:,0], X[:,1], c=y, zorder=10, cmap=plt.cm.Paired, edgecolors='k')
XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]
Z = clf.decision_function(np.c_[XX.ravel(),YY.ravel()])

# Put the results into the color plot
Z = Z.reshape(XX.shape)
plt.pcolormesh(XX,YY,Z>0,cmap=plt.cm.Paired)
plt.contour(XX, YY, Z, colors=['k','k','k'], linestyles=['--','-','--'], levels=[-.5,0,.5])
plt.xlim(x_min,x_max)
plt.ylim(y_min,y_max)

### **Polynomial Kernel**
A more interesting example is the polynomial kernel of degree $p$, of which we have already see a simple example.
$$K(x,z) = (x^\top z + c)^p.$$

This corresponds to a mapping to a feature space of dimension $d+p \choose p$ that has all monomials $x_{i_1}x_{i_2}\dots x_{i_p}$ of degree at most $p$.

For $d=3$ this features map looks like:

$$
\phi(x) = \begin{bmatrix}
x_1x_1 \\
x_1x_2 \\
x_1x_3 \\
x_2x_1 \\
x_2x_2 \\
x_2x_3 \\
x_3x_1 \\
x_3x_2 \\
x_3x_3 \\
\sqrt{2}cx_1 \\
\sqrt{2}cx_2 \\
\sqrt{2}cx_3 \\
c
\end{bmatrix}
$$

The polynomial kernel allows us to compute dot products in a $O(d^p)$-dimensional space in $O(d)$ times.

Let's see how it would be implement in `sklearn`

In [ ]:
clf = svm.SVC(kernel='poly', degree=3, gamma=2)
clf.fit(X,y)

# Plot the line, the points and the nearest vectors to the plane
plt.scatter(clf.support_vectors_[:,0], clf.support_vectors_[:,1], s=80, facecolors='none', edgecolors='k', zorder=10)
plt.scatter(X[:,0], X[:,1], c=y, zorder=10, cmap=plt.cm.Paired, edgecolors='k')
XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]
Z = clf.decision_function(np.c_[XX.ravel(),YY.ravel()])

# Put the results into the color plot
Z = Z.reshape(XX.shape)
plt.pcolormesh(XX,YY,Z>0,cmap=plt.cm.Paired)
plt.contour(XX, YY, Z, colors=['k','k','k'], linestyles=['--','-','--'], levels=[-.5,0,.5])
plt.xlim(x_min,x_max)
plt.ylim(y_min,y_max)

### **Radial Basis Function Kernel**

Another type is the **Radial Basis Function** (RBF; sometimes called Gaussian) kernel
$$K(x,z) = \exp\left(-\frac{\|x-z\|^2}{2\sigma^2}\right),$$

where $\sigma$ is the hyper-parameter. It's easiest to understand this by kernel by viewing it as a similarity measure.

We can show that this kernel corresponds to an **infinite-dimensional** feature map and the limit of the polynomial kernel as $p \to \infty$.

To see why that's intuitively the case, consider the Taylor expansion
$$\exp\left(-\frac{\|x-z\|^2}{2\sigma^2}\right) \approx 1 - \frac{\|x-z\|^2}{2\sigma^2} + \frac{\|x-z\|^4}{2!.4\sigma^4} - \frac{\|x-z\|^6}{3!.8\sigma^6} + \dots $$

Each term in the right hand side can be expanded into a polynomial.



We look at the `sklearn` implementation again

In [ ]:
clf = svm.SVC(kernel='rbf', gamma=.5)
clf.fit(X,y)

# Plot the line, the points and the nearest vectors to the plane
plt.scatter(clf.support_vectors_[:,0], clf.support_vectors_[:,1], s=80, facecolors='none', edgecolors='k', zorder=10)
plt.scatter(X[:,0], X[:,1], c=y, zorder=10, cmap=plt.cm.Paired, edgecolors='k')
XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]
Z = clf.decision_function(np.c_[XX.ravel(),YY.ravel()])

# Put the results into the color plot
Z = Z.reshape(XX.shape)
plt.pcolormesh(XX,YY,Z>0,cmap=plt.cm.Paired)
plt.contour(XX, YY, Z, colors=['k','k','k'], linestyles=['--','-','--'], levels=[-.5,0,.5])
plt.xlim(x_min,x_max)
plt.ylim(y_min,y_max)

### **When is $K$ a Kernel?**
We have seen that for many feature $\phi$ we can define a kernel function $K: \mathcal{X} \times \mathcal{X} \to [0,∞)$ that efficiently computes $\phi(x)^\top \phi(x).$

Suppose now that we use some kernel function $K: \mathcal{X} \times \mathcal{X} \to [0,∞)$ in an ML algorithm. Is there an implicit feature mapping $\phi$ that corresponds to using $K$?

Let's start by defining a necessary condition for $K: \mathcal{X} \times \mathcal{X} \to [0,∞)$ to associated with a feature map.



Suppose that $K$ is a kernel for some feature map $\phi$, and consider an arbitrary set of $n$ points $\{x^{(1)}, x^{(2)},\dots,x^{(n)}\}$.

Consider the matrix $L \in \mathbb{R}^{n \times n}$ defined as $L_{ij} = K(x^{(i)}, x^{(j)}) = \phi(x^{(i)})^\top\phi(x^{(j)})$. We claim that $L$ must be symmetric and positive semidefinite.

Indeed, $L$ is symmetric because the dot products $\phi(x^{(i)})^\top\phi(x^{(j)})$ is symmetric. Moreover, for any $z$:

\begin{align*}
z^\top Lz &= \sum_{i=1}^n\sum_{j=1}^n z_i L_{ij}z_j = \sum_{i=1}^n\sum_{j=1}^n z_i \phi(x^{(i)})^\top\phi(x^{(j)}) z_j \\
&= \sum_{i=1}^n\sum_{j=1}^n z_i \left(\sum_{k=1}^n \phi(x^{(i)})_k\phi(x^{(j)})_k\right) z_j \\
& = \sum_{i=1}^n\sum_{j=1}^n\sum_{k=1}^n z_i \phi(x^{(i)})_k\phi(x^{(j)})_k z_j \\
&= \sum_{k=1}^n\sum_{i=1}^n (z_i\phi(x^{(i)})_k )^2 \geq 0.
\end{align*}

Thus if $K$ is a Kernel, $L$ must be positive semidefinite for any $n$ points $x^{(i)}$.

### **Mercer's Theorem**

If $K$ is a Kernel, $L$ must be positive semidefinite for any $n$ points $x^{(i)}$. It turns out that it is also a sufficient condition:

**Theorem**(Mercer): Let $K: \mathcal{X} \times \mathcal{X} \to [0,\infty)$ be a kernel function. There exists a mapping feature $\phi$ associated with $K$ if for any $n$ and any dataset $\{x^{(i)}, x^{(2)}, \dots, x^{(n)}\}$ of size $n \geq 1$, if and only if the matrix $L$ defined as $L_{ij} = K(x^{(i)}, x^{(j)})$ is symmetric and positive semidefinite.

This characterizes precisely which kernel functions corresponds to some feature $\phi$.

### **Pros and Cons of Kernels**

Are Kernels a free lunch? Not quite:
* Kernels allow us to use feature $\phi$ of very large dimension $d$.
* Howeve computation is at least $O(n^2)$, where $n$ is the dataset size. We need to compute distances $K(x^{(i)}, x^{(j)})$ for all $i,j$.
* Approximate solutions can be found more quickly, but in practice kernel methods are not used with today's massive datasets.
* However, on small and medium-sized data, kernel method will be at least as good as neural nets and probably much easier to train.

### **Summary: Kernels**
* A kernel is a function $K: \mathcal{X} \times \mathcal{X} \to [0,\infty)$ that defines a notion of similarity over pairs of vectors in $\mathcal{X}$.
* Kernels are often associated with high-dimensional feature $\phi$ and inplicitly map inputs to this feature space.
* Kernels can be incorporated into many machine learning algorithms, which enables them to learn highly nonlinear models.

Examples of algorithms in which we can use kernel include:
* Supervised learning algorithms: linear regression, logistic regression, support vector machine, etc.
* Unsupervised learning algorithms: PCA, density estimation.

Kernels are very powerful because they can be used throughout machine learning.